In [1]:
using TuringModels
using Turing

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
size(d) # Should be 504x8

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24


(504, 8)

pulled_left, actors, condition, prosoc_left

In [2]:
@model m10_4(y, actors, x₁, x₂) = begin
    # Number of unique actors in the data set
    N_actor = length(unique(actors))
    # Set an TArray for the priors/param
    α = TArray{Any}(undef, N_actor)
    # For each actor [1,..,7] set a prior
    for i ∈ 1:length(α)
        α[i] ~ Normal(0,10)
    end

    βp ~ Normal(0, 10)
    βpC ~ Normal(0, 10)

    for i ∈ 1:length(y)
        p = logistic(α[actors[i]] + (βp + βpC * x₁[i]) * x₂[i])
        y[i] ~ Binomial(1, p)
    end
end;

posterior = sample(m10_4(d[:,:pulled_left], d[:,:actor],d[:,:condition],
d[:,:prosoc_left]), Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.8
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.09801634521936992, std = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 1039.8654113840005;
  #lf / sample        = 0.0;
  #evals / sample     = 67.273;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,....


Fix the inclusion of adaptation samples

In [3]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names);

Rethinking/CmdStan results

In [4]:
m_10_04s_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.74503184 0.26613979 0.0042080396 0.0060183398 1000
a.2 10.77955494 5.32538998 0.0842018089 0.1269148045 1000
a.3 -1.04982353 0.28535997 0.0045119373 0.0049074219 1000
a.4 -1.04898135 0.28129307 0.0044476339 0.0056325117 1000
a.5 -0.74390933 0.26949936 0.0042611590 0.0052178124 1000
a.6  0.21599365 0.26307574 0.0041595927 0.0045153523 1000
a.7  1.81090866 0.39318577 0.0062168129 0.0071483527 1000
 bp  0.83979926 0.26284676 0.0041559722 0.0059795826 1000
bpC -0.12913322 0.29935741 0.0047332562 0.0049519863 1000
 bpC  -0.13 0.30 -0.63  0.34  3508    1
";

Describe the draws

In [5]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                  SD                       Naive SE                      MCSE                ESS   
     βpC   -0.135010063  0.29465711501644087366714 0.0093178761222615560239069 0.0088045329947883405746767 1000.00000
  lf_num    0.000000000  0.00000000000000000000000 0.0000000000000000000000000 0.0000000000000000000000000        NaN
    α[2]    9.807223012  4.34796844204733012873021 0.1374948347140338422711636 0.4674209893971316143179706   86.52794
 epsilon    0.098016345  0.00000000000000008330839 0.0000000000000000026344427 0.0000000000000000046259293  324.32432
 elapsed    0.550549137  0.44476736547578293867744 0.0140647790381601325154781 0.0152051519620942394289687  855.62664
    α[4]   -1.052112766  0.28276508593773813649008 0.0089418171433650137785998 0.0084379483556627065538036 1000.00000
    α[1]   -0.744907626  0.27327472540482311558208 0.00864170559

End of 10/m_10_04t.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*